In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys

import pandas as pd
import numpy as np

import time

import torch.nn.functional as F
import torch

import copy

from tqdm.auto import tqdm

In [2]:
from datasets import load_original_dataset, load_deleted_dataset
from models import CNN

In [3]:
DATA_DIR = 'Datasets/Features/'
BATCH_SIZE = 32
EPOCHS = 5
PERCENTAGES = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99]

In [4]:
os.makedirs('results/unrolling-sgd', exist_ok=True)

# finetune

In [5]:
def fit(model, save_dir, train_set, test_set, forget_set):
    
    os.makedirs(save_dir, exist_ok=True)
    
    # prepare model
    
    # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/finetune.py#L94
    M = copy.deepcopy(model)
    
    # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/finetune.py#L150
    M_unlearned = copy.deepcopy(model)
        
    M_unlearned.train()
    
    # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/finetune.py#L131
    error = torch.nn.CrossEntropyLoss()
    
    forget_loader = torch.utils.data.DataLoader(forget_set, batch_size = BATCH_SIZE, shuffle = True, drop_last=True)
    
    train_times = list()
    train_accs, test_accs, forget_accs = list(), list(), list()
    
    for epoch in range(EPOCHS):    
        
        # train
        
        train_time = 0
        
        start_time = time.time()
        
        for x, y in forget_loader:

            output = M_unlearned(x.cuda())
            y = y.cuda()
            loss = error(output, y)
            
            # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/finetune.py#L163
            loss.backward(retain_graph=True)
            grads = torch.autograd.grad(loss, [param for param in M_unlearned.parameters()], create_graph = True)
            
            old_params = {}
            for i, (name, params) in enumerate(M.named_parameters()):
                    old_params[name] = params.clone()
                    old_params[name] += LR * grads[i]
            for name, params in M_unlearned.named_parameters():
                    params.data.copy_(old_params[name])
            
            train_time += time.time() - start_time
            
            start_time = time.time()
            
        train_times.append(train_time)
        
        # test
            
        M_unlearned.eval()
        with torch.no_grad():
            
            #
            
            x, y = train_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            train_accs.append(np.mean(accs))
            
            #
            
            x, y = test_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            test_accs.append(np.mean(accs))
            
            #

            x, y = forget_set.tensors
            
            accs = list()

            for i in range(0, x.shape[0], BATCH_SIZE):

                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())

            forget_accs.append(np.mean(accs))
        
        # save
        torch.save(M_unlearned.state_dict(), os.path.join(save_dir, f'{(epoch+1):03d}.pt'))

    return train_times, train_accs, test_accs, forget_accs

In [6]:
# https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/finetune.py#L49
LR = 0.1

In [7]:
results = list()

for percentage in tqdm(PERCENTAGES):
    
    model = CNN().cuda()
    
    model.load_state_dict(torch.load('./weights/original/005.pt'))
    
    train_set, test_set, forget_set = load_deleted_dataset(DATA_DIR, percentage)
    
    train_times, train_accs, test_accs, forget_accs = fit(model, f'weights/unrolling-sgd/finetune/{percentage}', train_set, test_set, forget_set)
    
    df = pd.DataFrame(zip(train_times, train_accs, test_accs, forget_accs), columns=['train_time', 'train_acc', 'test_acc', 'forget_acc'])
    df['epoch'] = range(1, EPOCHS+1)
    df['percentage'] = percentage
    
    results.append(df)

  0%|          | 0/11 [00:00<?, ?it/s]

In [8]:
results = pd.concat(results).set_index(['percentage', 'epoch'])

results.to_csv('results/unrolling-sgd/finetune.csv')

results

train_time  train_acc  test_acc  forget_acc
percentage epoch                                             
1          1        0.861320   0.098697  0.097943    0.097039
           2        0.149785   0.098697  0.097943    0.097039
           3        0.129887   0.098697  0.097943    0.097039
           4        0.141159   0.098697  0.097943    0.097039
           5        0.121428   0.098697  0.097943    0.097039
10         1        1.649805   0.098711  0.097943    0.098404
           2        1.710710   0.098711  0.097943    0.098404
           3        1.690747   0.098711  0.097943    0.098404
           4        1.691322   0.098711  0.097943    0.098404
           5        1.687659   0.098711  0.097943    0.098404
20         1        3.563996   0.098271  0.097943    0.100500
           2        3.589719   0.098271  0.097943    0.100500
           3        3.581025   0.098271  0.097943    0.100500
           4        3.538590   0.098271  0.097943    0.100500
           5        3.544239   0.098271  0.097943    0.100500
30         1        5.725345   0.098558  0.097943    0.099023
           2        5.367122   0.098558  0.097943    0.099023
           3        5.377095   0.098558  0.097943    0.099023
           4        5.430189   0.098558  0.097943    0.099023
           5        5.424914   0.098558  0.097943    0.099023
40         1        8.000167   0.098389  0.097943    0.099208
           2        7.510090   0.098389  0.097943    0.099208
           3        7.129795   0.098389  0.097943    0.099208
           4        7.136111   0.098389  0.097943    0.099208
           5        7.110609   0.098389  0.097943    0.099208
50         1       10.072822   0.098714  0.097943    0.098714
           2        9.602136   0.098714  0.097943    0.098714
           3        9.110337   0.098714  0.097943    0.098714
           4        9.109370   0.098714  0.097943    0.098714
           5        9.046206   0.098714  0.097943    0.098714
60         1       11.643538   0.099542  0.097943    0.098167
           2       10.602450   0.099542  0.097943    0.098167
           3       10.747267   0.099542  0.097943    0.098167
           4       10.938334   0.099542  0.097943    0.098167
           5       10.460525   0.099542  0.097943    0.098167
70         1       13.330676   0.098857  0.097943    0.098677
           2       12.482202   0.098857  0.097943    0.098677
           3       12.519452   0.098857  0.097943    0.098677
           4       13.007364   0.098857  0.097943    0.098677
           5       12.460160   0.098857  0.097943    0.098677
80         1       15.516491   0.099500  0.097943    0.098521
           2       14.571281   0.099500  0.097943    0.098521
           3       14.648647   0.099500  0.097943    0.098521
           4       16.046360   0.099500  0.097943    0.098521
           5       14.730189   0.099500  0.097943    0.098521
90         1       18.407147   0.097739  0.097943    0.098804
           2       17.392190   0.097739  0.097943    0.098804
           3       17.713189   0.097739  0.097943    0.098804
           4       16.665290   0.097739  0.097943    0.098804
           5       16.683773   0.097739  0.097943    0.098804
99         1       20.123055   0.095943  0.097943    0.098697
           2       18.979310   0.095943  0.097943    0.098697
           3       18.767822   0.095943  0.097943    0.098697
           4       18.995738   0.095943  0.097943    0.098697
           5       18.159758   0.095943  0.097943    0.098697

# correlation

In [9]:
def std_loss(x, y):
    log_prob = -1.0 * F.log_softmax(x, 1)
    loss = log_prob.gather(1, y.unsqueeze(1))
    loss = loss.mean()
    avg_std = torch.sum(torch.std(x, dim=1))/(len(x.view(-1)))
    loss = loss + STD_REG*avg_std
    return loss

def my_cross_entropy(x, y):
    log_prob = -1.0 * F.log_softmax(x, 1)
    loss = log_prob.gather(1, y.unsqueeze(1))
    loss = loss.mean()

    #x is (N,C)
    N,C = x.shape
    p = F.softmax(x,1)
    hessian_loss = torch.sum(p *(1-p),dim =1)
    hessian_loss = hessian_loss.mean()

    loss = loss + STD_REG * hessian_loss
    return loss

# https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L130

loss_fn = torch.nn.CrossEntropyLoss()

In [10]:
def fit(model, save_dir, train_set, test_set, forget_set):
    
    os.makedirs(save_dir, exist_ok=True)
    
    # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L161

    M = copy.deepcopy(model)

    M.train()
    
    # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L172
    optimizer = torch.optim.SGD(M.parameters(), lr=LR, weight_decay = L2)
    
    
    train_loader = torch.utils.data.DataLoader(train_set, batch_size = BATCH_SIZE, shuffle = True, drop_last=True)
    
    forget_loader = torch.utils.data.DataLoader(forget_set, batch_size = BATCH_SIZE, shuffle = True, drop_last=True)
    
    train_times = list()
    train_accs, test_accs, forget_accs = list(), list(), list()
    
    for epoch in range(EPOCHS):
        
        # train
        
        train_time = 0
        
        start_time = time.time()
        
        # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L200
        
        # only update M:
        
        M.train()
        
        for x, y in forget_loader:
        
            optimizer.zero_grad()
            
            outputs_M = M(x.cuda())
            
            targets = y.cuda()
        
            if LOSS_FUNC == 'hess':
                loss_M = my_cross_entropy(outputs_M, targets)
            if LOSS_FUNC =='std':
                loss_M = std_loss(outputs_M, targets)
            if LOSS_FUNC =='regular':
                loss_M = loss_fn(outputs_M, targets)
                
            loss_M.backward()
            optimizer.step()
        
        # now we also want to compute the gradient:
        
        grad_list = []
        
        for x, y in train_loader:

            output_grad = M(x.cuda())
            
            label = y.cuda()
            
            if LOSS_FUNC == 'hess':
                loss_grad = my_cross_entropy(output_grad, label)
            if LOSS_FUNC =='std':
                loss_grad = std_loss(output_grad, label)
            if LOSS_FUNC =='regular':
                loss_grad = loss_fn(output_grad, label)
            
            loss_grad.backward(retain_graph=True)
            grads = torch.autograd.grad(loss_grad, [param for param in M.parameters()], create_graph = True)
            # !!! modified !!! original code cause OOM
            # grad_list.append(grads)
            grad_list.append([i.detach().cpu() for i in grads])
        
        # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L227
            
        # update both M and M'
        
        for x, y in forget_loader:
        
            optimizer.zero_grad()
            
            outputs_M = M(x.cuda())
            
            targets = y.cuda()
        
            if LOSS_FUNC == 'hess':
                loss_M = my_cross_entropy(outputs_M, targets)
            if LOSS_FUNC =='std':
                loss_M = std_loss(outputs_M, targets)
            if LOSS_FUNC =='regular':
                loss_M = loss_fn(outputs_M, targets)
                
            loss_M.backward()
            optimizer.step()
            
        # Now, get M''_(N+t)
            
        # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L270
        
        M_unlearned = copy.deepcopy(M)
        
        old_params = {}
        for i, (name, params) in enumerate(M.named_parameters()):
            old_params[name] = params.clone()
            for grads in grad_list:
                old_params[name] += LR * grads[i].cuda()
        for name, params in M_unlearned.named_parameters():
            params.data.copy_(old_params[name])
        
            
        train_time += time.time() - start_time

        start_time = time.time()
            
        train_times.append(train_time)
        
        # test
            
        M_unlearned.eval()
        with torch.no_grad():
            
            #
            
            x, y = train_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            train_accs.append(np.mean(accs))
            
            #
            
            x, y = test_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            test_accs.append(np.mean(accs))
            
            #

            x, y = forget_set.tensors
            
            accs = list()

            for i in range(0, x.shape[0], BATCH_SIZE):

                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())

            forget_accs.append(np.mean(accs))
        
        # save
        torch.save(M_unlearned.state_dict(), os.path.join(save_dir, f'{(epoch+1):03d}.pt'))

    return train_times, train_accs, test_accs, forget_accs

In [11]:
# https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L167
LR = 0.01
# https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L72
L2 = 0.
# https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L71
STD_REG = 0.

LOSS_FUNC = 'hess'

In [12]:
results = list()

for percentage in tqdm(PERCENTAGES):
    
    model = CNN().cuda()
    
    model.load_state_dict(torch.load('./weights/original/005.pt'))
    
    train_set, test_set, forget_set = load_deleted_dataset(DATA_DIR, percentage)
    
    train_times, train_accs, test_accs, forget_accs = fit(model, f'weights/unrolling-sgd/correlation/{percentage}', train_set, test_set, forget_set)
    
    df = pd.DataFrame(zip(train_times, train_accs, test_accs, forget_accs), columns=['train_time', 'train_acc', 'test_acc', 'forget_acc'])
    df['epoch'] = range(1, EPOCHS+1)
    df['percentage'] = percentage
    
    results.append(df)

  0%|          | 0/11 [00:00<?, ?it/s]

In [13]:
results = pd.concat(results).set_index(['percentage', 'epoch'])

results.to_csv('results/unrolling-sgd/correlation.csv')

results

train_time  train_acc  test_acc  forget_acc
percentage epoch                                             
1          1       25.764190   0.104385  0.102736    0.103070
           2       23.685421   0.112177  0.113518    0.131031
           3       21.660932   0.108879  0.115615    0.113487
           4       22.418080   0.097721  0.099541    0.107456
           5       22.731835   0.090468  0.089257    0.084430
10         1       21.352592   0.097267  0.098942    0.104721
           2       21.849211   0.098341  0.095747    0.101729
           3       21.849150   0.111708  0.112919    0.111868
           4       21.583047   0.102247  0.101038    0.101230
           5       24.558183   0.103951  0.102736    0.108544
20         1       24.163665   0.103937  0.101138    0.098250
           2       22.754565   0.102146  0.101038    0.102333
           3       22.885594   0.073292  0.074281    0.065917
           4       23.689296   0.099583  0.100739    0.099667
           5       22.732578   0.229479  0.235823    0.231083
30         1       22.394233   0.091299  0.089257    0.088088
           2       22.529638   0.134592  0.129892    0.132549
           3       22.501310   0.188619  0.185703    0.185058
           4       22.656119   0.064666  0.062899    0.067607
           5       23.018073   0.144112  0.146066    0.142651
40         1       24.329224   0.104889  0.102736    0.103708
           2       23.939088   0.098611  0.101737    0.100708
           3       23.193644   0.098278  0.100839    0.100458
           4       23.647509   0.111556  0.113518    0.114042
           5       23.828699   0.116167  0.109625    0.115667
50         1       25.336387   0.122668  0.129293    0.127265
           2       23.750583   0.109841  0.115016    0.109375
           3       23.896306   0.153851  0.154852    0.152452
           4       24.098047   0.105244  0.102736    0.103578
           5       23.947850   0.116671  0.120208    0.120502
60         1       23.459184   0.096875  0.100839    0.100667
           2       23.363450   0.108042  0.115216    0.112222
           3       23.397421   0.123167  0.121506    0.118639
           4       23.583306   0.256125  0.258786    0.259000
           5       24.104743   0.146750  0.149361    0.144306
70         1       24.747838   0.096470  0.099740    0.100819
           2       23.775182   0.103464  0.101038    0.101604
           3       23.917100   0.107960  0.113918    0.114314
           4       23.781101   0.133659  0.125300    0.128332
           5       23.847407   0.189443  0.191693    0.194664
80         1       24.722005   0.095083  0.091254    0.093396
           2       24.991220   0.097500  0.095847    0.099521
           3       24.967969   0.251583  0.246206    0.242125
           4       24.173449   0.556833  0.566094    0.561292
           5       24.717310   0.472333  0.482129    0.477396
90         1       25.163023   0.644614  0.640875    0.639977
           2       26.689583   0.428856  0.431310    0.422375
           3       25.583513   0.390459  0.386981    0.386423
           4       24.881740   0.870346  0.870108    0.870761
           5       25.125331   0.768949  0.772364    0.771438
99         1       25.574194   0.970395  0.968650    0.968127
           2       25.713386   0.965461  0.969848    0.969608
           3       25.140595   0.981908  0.987520    0.987850
           4       25.816537   0.983553  0.985823    0.987396
           5       25.232232   0.988487  0.988319    0.987934